In [35]:
import websocket
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
from code_ref import get_keys
from code_ref import findata
from code_ref import FinnhubRequestException
from code_ref import FinnhubAPIException
import time
import threading
%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
keys = get_keys("/Users/augustingoudet/.secret/code_file") #Finnhub Key
api_key1 = keys ['finnhub'] #api_key
company = "SBUX" #Company Ticker

query = findata(api_key = api_key1)


In [42]:
url = "https://finnhub.io/api/v1/"

ext1 = "?symbol={}&token={}"
ext2 = "?symbol={}&metric=management&token={}"
ext3 = "?symbol={}&metric=margin&token={}"
ext4 = "?symbol={}&metric=growth&token={}"
ext5 = "?symbol={}&metric=price&token={}"

profile ="stock/profile"
ceo_comp = "stock/ceo-compensation"
board = "stock/executive"
reco = "stock/recommendation"
price_target = "stock/price-target"
up_down = "stock/upgrade-downgrade"
peers = "stock/peers"
rev_est = "stock/revenue-estimate"
eps_est = "stock/eps-estimate"
ownership = "stock/investor-ownership"
metric_1 = "stock/metric"
quote = "quote"
news = "news"
covid = "covid19"
transcript = "stock/transcripts"
divs = "stock/dividend?symbol={}&from=2019-02-01&to=2020-02-01"
split = "stock/split?symbol=AAPL&from=2010-02-01&to=2020-02-01"
inc = "stock/financials?symbol={}&statement=is&freq=annual"
bs = "stock/financials?symbol={}&statement=bs&freq=annual"
cfs = "stock/financials?symbol={}&statement=cf&freq=annual"


In [38]:
tickers = pd.read_csv('Ticker_list_1.csv')

In [39]:
tick = list(tickers.Ticker)
tick1 = tick[:7]

## Profiling Data

In [40]:
results = []

for items in tick1: 
    data = query.ceo_compensation(symbol = items)
    results.append(data)
    df = pd.DataFrame(results)
    df.dropna(inplace = True)


%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 13.1 µs


In [41]:
df

,bonus,companyName,location,name,nonEquityIncentives,optionAwards,otherComp,pensionAndDeferred,salary,stockAwards,symbol,total,year
1,0.0,Apple Inc.,"Cupertino, CA",Timothy Cook,12000000.0,0.0,682219.0,0.0,3000000.0,0.0,AAPL,15682219.0,2018
2,0.0,Bank of America Corp,"Charlotte, NC",Brian Moynihan,0.0,0.0,382263.0,320159.0,1500000.0,20552088.0,BAC,22754510.0,2018
3,0.0,Amazon.com Inc.,"Seattle, WA",Jeffrey Bezos,0.0,0.0,1600000.0,0.0,81840.0,0.0,AMZN,1681840.0,2018
4,0.0,AT&T Inc.,"Dallas, TX",Randall Stephenson,5192000.0,0.0,1538538.0,3517806.0,1800000.0,17069774.0,T,29118118.0,2018
6,0.0,Altria Group Inc,"Richmond, VA",Martin Barrington,833300.0,0.0,175665.0,1505621.0,684816.0,0.0,MO,3199402.0,2018


## board members / executive

In [59]:
results = []

for items in tick1: 
    data = url + ceo_comp + ext1.format(items, api_key1)
    req = requests.get(data)
    content = req.json()
    results.append(content)
    df1 = pd.DataFrame(results)
    df1.dropna(inplace = True)

In [60]:
df1

,bonus,companyName,location,name,nonEquityIncentives,optionAwards,otherComp,pensionAndDeferred,salary,stockAwards,symbol,total,year
1,0.0,Apple Inc.,"Cupertino, CA",Timothy Cook,12000000.0,0.0,682219.0,0.0,3000000.0,0.0,AAPL,15682219.0,2018
2,0.0,Bank of America Corp,"Charlotte, NC",Brian Moynihan,0.0,0.0,382263.0,320159.0,1500000.0,20552088.0,BAC,22754510.0,2018
3,0.0,Amazon.com Inc.,"Seattle, WA",Jeffrey Bezos,0.0,0.0,1600000.0,0.0,81840.0,0.0,AMZN,1681840.0,2018
4,0.0,AT&T Inc.,"Dallas, TX",Randall Stephenson,5192000.0,0.0,1538538.0,3517806.0,1800000.0,17069774.0,T,29118118.0,2018
6,0.0,Altria Group Inc,"Richmond, VA",Martin Barrington,833300.0,0.0,175665.0,1505621.0,684816.0,0.0,MO,3199402.0,2018


## analyst recommendation 

In [68]:
pd.DataFrame(query.earnings(symbol = "SBUX"))

,actual,estimate,period,symbol
0,0.79000,0.75864,2019-12-31,SBUX
1,0.70000,0.69867,2019-09-30,SBUX
2,0.78000,0.72407,2019-06-30,SBUX
3,0.60000,0.56313,2019-03-31,SBUX
4,0.68000,0.64839,2018-12-31,SBUX
...,...,...,...,...
72,0.03750,0.03684,2001-12-31,SBUX
73,0.03500,0.03334,2001-09-30,SBUX
74,0.03000,0.03035,2001-06-30,SBUX
75,0.02000,0.01986,2001-03-31,SBUX


In [77]:
results = []

for items in tick1: 
    data = query.earnings(symbol = items)
    earnings = pd.DataFrame(data)  

In [78]:
earnings

,actual,estimate,period,symbol
0,1.02,1.01908,2019-12-31,MO
1,1.19,1.14892,2019-09-30,MO
2,1.10,1.10571,2019-06-30,MO
3,0.90,0.92179,2019-03-31,MO
4,0.95,0.94986,2018-12-31,MO
...,...,...,...,...
67,1.07,1.06000,2003-03-31,MO
68,0.93,0.92444,2002-12-31,MO
69,1.26,1.26000,2002-09-30,MO
70,1.24,1.22875,2002-06-30,MO


## price target

In [ ]:
url3 = "https://finnhub.io/api/v1/stock/price-target?symbol={}&token={}".format(company, api_key) #url
req3 = requests.get(url3) 

In [ ]:
print(req3.json())

## Stock Upgrade / Downgrade

In [ ]:
url4 = "https://finnhub.io/api/v1/stock/upgrade-downgrade?symbol={}&token={}".format(company, api_key) #url
req4 = requests.get(url4) 
status = pd.DataFrame(req4.json())

In [ ]:
status

### Peers

In [ ]:
url5 = "https://finnhub.io/api/v1/stock/peers?symbol={}&token={}".format(company, api_key) #url
req5 = requests.get(url5) 

In [ ]:
print(req5.json())

## Revenue Estimates

In [ ]:
url6 = "https://finnhub.io/api/v1/stock/revenue-estimate?symbol={}&token={}".format(company, api_key) #url
req6 = requests.get(url6) 
rev = pd.DataFrame(req6.json())
rev1 = json_normalize(rev['data'])
rev1

## Earnings Estimate

In [ ]:
url7 = "https://finnhub.io/api/v1/stock/eps-estimate?symbol={}&token={}".format(company, api_key) #url
req7 = requests.get(url7) 
print(req7.json())

In [ ]:
eps = pd.DataFrame(req7.json())
eps1 = json_normalize(eps['data'])
eps1

## Investor Ownership

In [ ]:
url8 = "https://finnhub.io/api/v1/stock/investor-ownership?symbol={}&token={}".format(company, api_key) #url
req8 = requests.get(url8) 
# print(req8.json())
own = req8.json()

In [ ]:
ownership = pd.DataFrame(own.items(), columns =['Features', 'Value'])
# print(req8.json())
owner = pd.DataFrame(req8.json())
owner1 = json_normalize(owner['ownership'])

In [ ]:
owner1

## Fund Ownership

In [ ]:
url9 = "https://finnhub.io/api/v1/stock/fund-ownership?symbol={}&token={}".format(company, api_key) #url
req9 = requests.get(url9)
fdowner = pd.DataFrame(req9.json())
fdowner1 = json_normalize(fdowner['ownership'])
fdowner1

## performance metrics

In [ ]:
url9 = "https://finnhub.io/api/v1/stock/metric?symbol={}&metric=margin&token={}".format(company, api_key) #url
req9 = requests.get(url9) 
print(req9.json())

In [ ]:
perf = pd.DataFrame(req9.json())
perf

## stock quote

In [ ]:
url10 = "https://finnhub.io/api/v1/quote?symbol={}&token={}".format(company, api_key) #url
req10 = requests.get(url10) 
print(req10.json())

In [ ]:
url10 = "https://finnhub.io/api/v1/quote?symbol=BINANCE:BTCEUR&token={}".format(api_key) #url
req10 = requests.get(url10) 
print(req10.json())

## General News

In [ ]:
url11 = "https://finnhub.io/api/v1/news?category=general&token={}".format(api_key) #url
req11 = requests.get(url11) 
news = pd.DataFrame(req11.json())
news

## company news 

In [ ]:
url12 = "https://finnhub.io/api/v1/news/{}?token={}".format(company,api_key) #url
req12 = requests.get(url12) 
compnews = pd.DataFrame(req12.json())
compnews

## Major Developments

In [ ]:
url13 = "https://finnhub.io/api/v1/major-development?symbol={}&token={}'".format(company,api_key) #url
req13 = requests.get(url13) 
# print(req13.json())

## New Sentiment

In [ ]:
url14 = "https://finnhub.io/api/v1/news-sentiment?symbol={}&token={}".format(company,api_key) #url
req14 = requests.get(url14) 

cpn = pd.DataFrame(req14.json())
print(req14.json())

## Economic Code & Data


In [ ]:
url15 = "https://finnhub.io/api/v1/economic/code?token={}".format(api_key) #url
req15 = requests.get(url15) 
# print(req15.json())


In [ ]:
code = pd.DataFrame(req15.json())
code.head(n=15)

In [ ]:
df = code.loc[code['country'] == 'United States']

In [ ]:
code.code.value_counts().sum()

In [ ]:
url16 = "https://finnhub.io/api/v1/economic?code=A-FRA-6682&token={}".format(api_key) #url
req16 = requests.get(url16) 
eco = pd.DataFrame(req16.json(), columns =['Date', 'Monthly Bankruptices in France'])
eco

In [ ]:
url17 = "https://finnhub.io/api/v1/covid19/us?token={}".format(api_key) #url
req17 = requests.get(url17) 
# print(req17.json())

covid = pd.DataFrame(req17.json())
covid.head(n=20)